## Configurando o MySQL com Python

Neste momento, vamos configurar a integração entre o MySQL e Python. Isso nos permitirá executar consultas, inserções e outras operações no banco de dados MySQL usando o Python como nossa linguagem de programação principal.

### Obs
- Apesar de o curso estar estruturado em MySql, daremos sequência aqui com a integração ao **SQL Server**, que é mais familiar e confortável

In [5]:
import pyodbc 

try:
    print('Tentando conectar ao banco Sql Server...')
    conn = pyodbc.connect('DRIVER=SQL Server;Server=DESKTOP-E29RO7M\\SQLEXPRESS;Database=dbprodutos')
    print('Conexão bem-sucedida!')
except Exception as e:
    print('A conexão falhou.')
    print(e)


Tentando conectar ao banco Sql Server...
Conexão bem-sucedida!


Criaremos um cursor, que é um objeto utilizado para executar instruções SQL no contexto do Python. Essse cursor nos permitirá enviar consultas e comandos SQL para o banco de dados MySQL por meio da conexão estabelecida e obter os resultados de volta para serem processados em nosso código Python

In [6]:
# definindo o objeto cursor (responsável por permitir a manipualação de tabelas e registros no banco de dados através do python)
cursor = conn.cursor()

## Criando uma base de dados

O método `execute()` é usado para compilar uma instrução SQL

In [7]:
for db in cursor.execute('''
                         SELECT name, database_id, create_date  
                            FROM sys.databases
'''):
    print(db.name)

master
tempdb
model
msdb
Lista 1 - Exercícios
SQL_DB_1
Universidade-Exercício
AdventureWorks
ProjetosFrontEnd
CursoSqlAlchemy
dbprodutos


## Criando uma tabela

Antes de criar nossa tabela, vamos revisar as colunas necessárias que precisaremos incluir nela. Essa etapa é importante para garantir que nossa tabela seja projetada corretamente, atendendo aos requisitos de armazenamento e organização dos dados.

In [8]:
import pandas as pd 

df_livros = pd.read_csv('./data/tabela_livros.csv')
display(df_livros)

,_id,Produto,Categoria do Produto,Preço,Frete,Data da Compra,Vendedor,Local da compra,Avaliação da compra,Tipo de pagamento,Quantidade de parcelas,Latitude,Longitude
0,667f3fd483e95f42cebda945,Modelagem preditiva,livros,92.45,5.609697,2020-01-01,Thiago Silva,BA,1,cartao_credito,3,-13.29,-41.71
1,667f3fd483e95f42cebda946,Iniciando em programação,livros,43.84,0.000000,2020-01-01,Mariana Ferreira,SP,5,cartao_credito,1,-22.19,-48.79
2,667f3fd483e95f42cebda958,Iniciando em programação,livros,63.25,3.894137,2022-01-01,Juliana Costa,RJ,5,cartao_credito,4,-22.25,-42.66
3,667f3fd483e95f42cebda966,Ciência de dados com python,livros,86.13,5.273176,2021-02-01,Camila Ribeiro,RJ,4,cartao_credito,3,-22.25,-42.66
4,667f3fd483e95f42cebda968,Ciência de dados com python,livros,72.75,1.458158,2021-02-01,Beatriz Moraes,PR,4,cartao_credito,2,-24.89,-51.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...
737,667f3fd483e95f42cebdcde0,Dashboards com Power BI,livros,41.27,2.728608,2022-07-31,João Souza,SC,5,cartao_credito,12,-27.45,-50.95
738,667f3fd483e95f42cebdcde4,Ciência de dados com python,livros,81.58,4.632666,2020-08-31,Juliana Costa,SP,5,cartao_credito,2,-22.19,-48.79
739,667f3fd483e95f42cebdcdea,Modelagem preditiva,livros,109.17,6.701389,2021-08-31,Felipe Santos,MG,3,cartao_credito,3,-18.10,-44.38
740,667f3fd483e95f42cebdcdf6,Modelagem preditiva,livros,94.89,5.578872,2022-08-31,Pedro Gomes,SP,5,cartao_credito,1,-22.19,-48.79


In [10]:
df_livros.columns

Index(['_id', 'Produto', 'Categoria do Produto', 'Preço', 'Frete',
       'Data da Compra', 'Vendedor', 'Local da compra', 'Avaliação da compra',
       'Tipo de pagamento', 'Quantidade de parcelas', 'Latitude', 'Longitude'],
      dtype='object')

In [11]:
df_livros.shape

(742, 13)

In [15]:
cursor.execute('''
    create table TabelaLivros 
    (
        Id varchar(1000) primary key,
        Produto varchar(1000),
        CategoriaProduto varchar(1000),
        Preco decimal(18,2),
        Frete decimal(18,6),
        DataCompra date,
        Vendedor varchar(1000),
        LocalCompra varchar(1000),
        AvaliacaoCompra int,
        TipoPagamento varchar(1000),
        QtdParcelas int,
        Latitude decimal(18,2),
        Longitude decimal(18,2)
    )
''')
cursor.commit()

**Selecionando a base de dados para verificar a tabela criada**

In [20]:
cursor.execute("select distinct Name from sys.Tables")
for item in cursor:
    print(item)

('TabelaLivros',)


## Inserindo os dados do csv na tabela

Para inserir os dados na tabela do MySQL, é necessário percorrer cada linha do DataFrame e transformá-las em tuplas. Essa abordagem permite que mapeemos os dados do DataFrame para as colunas correspondentes da tabela do MySQL de forma eficiente e precisa.

In [25]:
for i, row in df_livros.iterrows():
    if i in [0,1,2,3,4]:
        print(tuple(row))

('667f3fd483e95f42cebda945', 'Modelagem preditiva', 'livros', 92.45, 5.6096965236, '2020-01-01', 'Thiago Silva', 'BA', 1, 'cartao_credito', 3, -13.29, -41.71)
('667f3fd483e95f42cebda946', 'Iniciando em programação', 'livros', 43.84, 0.0, '2020-01-01', 'Mariana Ferreira', 'SP', 5, 'cartao_credito', 1, -22.19, -48.79)
('667f3fd483e95f42cebda958', 'Iniciando em programação', 'livros', 63.25, 3.894137208, '2022-01-01', 'Juliana Costa', 'RJ', 5, 'cartao_credito', 4, -22.25, -42.66)
('667f3fd483e95f42cebda966', 'Ciência de dados com python', 'livros', 86.13, 5.2731755368, '2021-02-01', 'Camila Ribeiro', 'RJ', 4, 'cartao_credito', 3, -22.25, -42.66)
('667f3fd483e95f42cebda968', 'Ciência de dados com python', 'livros', 72.75, 1.458157705, '2021-02-01', 'Beatriz Moraes', 'PR', 4, 'cartao_credito', 2, -24.89, -51.55)


In [26]:
lista_registros = [tuple(row) for i, row in df_livros.iterrows()]

In [29]:
sql_query = '''
    insert into TabelaLivros (
        Id
        , Produto
        , CategoriaProduto
        , Preco
        , Frete
        , DataCompra
        , Vendedor
        , LocalCompra
        , AvaliacaoCompra
        , TipoPagamento
        , QtdParcelas
        , Latitude
        , Longitude
        ) 
    values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)'''
# os ? representam parâmetros dinâmicos no contexto dessa conexão (via PyOdbc)

cursor.executemany(sql_query, lista_registros)
cursor.commit()

In [35]:
a = cursor.execute('select count(*) from TabelaLivros').fetchall() # o fetchall converte o objeto do tipo cursor em uma lista de tuplas (cada linha da consulta se torna uma tupla)
print(a)

[(742,)]


In [36]:
print(cursor.rowcount)

TypeError: 'int' object is not callable

## Visualizando os dados inseridos